In [2]:
#!/usr/bin/python3
from binance.client import Client
from time import time
import pickle as pickle
from datetime import datetime
import pandas as pd
import pandas_ta as ta
from pycaret.classification import load_model, predict_model
from pycaret.classification import *
from pycaret.classification import ClassificationExperiment
from lightgbm import LGBMClassifier
import numpy as np
import random


def get_client():
    fn = '../../key/binance-key.pickle'
    # fn = '/home/era/key/binance-key.pickle'
    with open(fn, 'rb') as handle:
        k = pickle.load(handle)
    return Client(k['API_KEY'], k['API_SECRET'])


client = get_client()


def get_unix_timestamp(date_string):
    """
    Converts the input date string to Unix timestamp.

    Parameters:
        date_string (str): Input date string in the format "dd/mm/yyyy hh:mm:ss".

    Returns:
        int: Unix timestamp of the given date.
    """
    try:
        date_obj = datetime.strptime(date_string, "%d/%m/%Y %H:%M:%S")
        timestamp = int(date_obj.timestamp())
        return timestamp
    except ValueError:
        print("Invalid date format. Please use the format 'dd/mm/yyyy hh:mm:ss'.")
        return None

def get_historical_data(start_timestamp, end_timestamp, coin_pair): 
    data = []
    tot = (end_timestamp - start_timestamp)/(900*500)
    cntr = 0
    for current_sts in range(start_timestamp, end_timestamp+1, 900*500):
        next_ets = current_sts + 900*500 if (current_sts + 900*500) < end_timestamp else end_timestamp
        print(current_sts, next_ets, f'100% completed') if next_ets == end_timestamp else print(current_sts, next_ets, f'{round(cntr*100/tot, 1)}% completed')
        cntr += 1
        # Futures market
        klines = client.futures_historical_klines(coin_pair, '15m', current_sts*1000, next_ets*1000, limit=500)
        # Spot market
        # klines = client.get_historical_klines(coin_pair, interval, current_sts*1000, next_ets*1000, limit=500)
        
        for kline in klines:
            timestamp = kline[0]/1000
            open_price = float(kline[1])
            high_price = float(kline[2])
            low_price = float(kline[3])
            close_price = float(kline[4])
            volume = float(kline[5])

            data.append([timestamp, open_price, high_price, low_price, close_price, volume])

    df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
    # df.to_csv(f'/home/ubuntu/data/{coin_pair}-{interval}.csv', index=False)
    # print('Data Exported')
    print(f'Historical Data of {coin_pair} Downloaded')
    return df

def generate_features(start_timestamp, end_timestamp, coin_pair):
    df = get_historical_data(start_timestamp, end_timestamp, coin_pair)
    candlestick_frame = 12
    pnl_threshold = 3


    # df = pd.read_csv('/home/ubuntu/data/ETHUSDT-15m.csv')
    try:
        df.ta.strategy("all")
    except Exception as e:
        print(e)
        ts = list(df['time'])
        open = list(df['open'])
        high = list(df['high'])
        low = list(df['low'])
        close = list(df['close'])
        volume = list(df['volume'])
        tot = len(ts)
        long_runup_lst = []
        long_drawdown_lst = []
        short_runup_lst = []
        short_drawdown_lst = []

        for idx in range(tot):
            if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
                max_high = max(high[idx+1:idx+candlestick_frame])
                min_low = min(low[idx+1:idx+candlestick_frame])
                entry_price = open[idx+1]
                long_runup_lst.append(round((max_high*100/entry_price)-100, 6))
                long_drawdown_lst.append(round((min_low*100/entry_price)-100, 6))
                short_runup_lst.append(round((entry_price*100/min_low)-100, 6))
                short_drawdown_lst.append(round((entry_price*100/max_high)-100, 6))
            else:
                long_runup_lst.append(0)
                long_drawdown_lst.append(0)
                short_runup_lst.append(0)
                short_drawdown_lst.append(0)     


        long=[]
        short=[]
        dont_trade=[]
        signal = []

        for idx in range(tot):
            if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
                if long_runup_lst[idx] >= pnl_threshold:
                    signal.append('long')
                elif short_runup_lst[idx] >= pnl_threshold:
                    signal.append('short')
                else:
                    signal.append('dont_trade')
            else:
                signal.append('dont_trade')

        df['coin'] = [coin_pair]*len(signal)
        df['signal'] = signal

        long_indices = df[df['signal'].str.contains('long', case=False)].index
        short_indices = df[df['signal'].str.contains('short', case=False)].index
        dont_trade_indices = list(df[df['signal'].str.contains('dont_trade', case=False)].index)
        num_indices_to_pick  = len(dont_trade_indices) - min([len(long_indices), len(short_indices)])
        random_indices = random.sample(dont_trade_indices, num_indices_to_pick)
        df = df.drop(random_indices)

        df.reset_index(drop=True, inplace=True)

        df = df.drop(columns = ['time', 'open', 'high', 'low', 'close', 'volume'], axis=1)
        dataset_fn = f'../../data/{coin_pair}-dataset.csv'
        pd.DataFrame(df).to_csv(dataset_fn, index=False)
        print(f'{coin_pair} Features Generated and saved')


# coins_list = ['BTC', 'ETH', 'BNB', 'XRP', 'ADA', 'DOGE', 'SOL', 'TRX', 'DOT', 'MATIC', 'LTC', 'BCH', 'AVAX', 'XLM', 'LINK', 'UNI', 'XMR', 'ATOM', 'ETC', 'HBAR', 'ICP', 'FIL', 'LDO', 'APT', 'ARB', 'QNT', 'VET', 'NEAR', 'OP', 'MKR', 'GRT', 'AAVE', 'ALGO', 'AXS', 'EGLD', 'STX', 'SAND', 'XTZ', 'EOS', 'INJ', 'THETA', 'IMX', 'SNX', 'MANA', 'FTM', 'RUNE', 'APE', 'RNDR', 'NEO', 'KAVA', 'FLOW', 'CHZ', 'GALA', 'KLAY', 'SUI', 'FXS', 'ZEC', 'CFX', 'CRV', 'MINA', 'COMP', 'GMX', 'DYDX', 'WOO', 'ASTR']
coins_list = ['ETH']

for i in range(len(coins_list)):
    coin_pair = f'{coins_list[i]}USDT'
    print('working on', coin_pair)
    # start_timestamp = get_unix_timestamp('1/1/2016 00:00:00')
    start_timestamp = get_unix_timestamp('1/6/2022 00:00:00')
    end_timestamp = int(time())
    generate_features(start_timestamp, end_timestamp, coin_pair)
    
coin_feaures = [f'{val}USDT-dataset.csv' for val in coins_list]
dfs=[]
dataset_path = '../../data/'

for fn in coin_feaures:
    dfs.append(pd.read_csv(f'{dataset_path}{fn}'))

concatenated_df = pd.concat(dfs, ignore_index=True)  # Set ignore_index=True to reset index
pd.DataFrame(concatenated_df).to_csv(f'{dataset_path}combined_dataset.csv', index=False)


working on ETHUSDT
1654027200 1654477200 0.0% completed
1654477200 1654927200 1.1% completed
1654927200 1655377200 2.2% completed
1655377200 1655827200 3.3% completed
1655827200 1656277200 4.4% completed
1656277200 1656727200 5.5% completed
1656727200 1657177200 6.6% completed
1657177200 1657627200 7.7% completed
1657627200 1658077200 8.8% completed
1658077200 1658527200 9.9% completed
1658527200 1658977200 11.0% completed
1658977200 1659427200 12.1% completed
1659427200 1659877200 13.2% completed
1659877200 1660327200 14.4% completed
1660327200 1660777200 15.5% completed
1660777200 1661227200 16.6% completed
1661227200 1661677200 17.7% completed
1661677200 1662127200 18.8% completed
1662127200 1662577200 19.9% completed
1662577200 1663027200 21.0% completed
1663027200 1663477200 22.1% completed
1663477200 1663927200 23.2% completed
1663927200 1664377200 24.3% completed
1664377200 1664827200 25.4% completed
1664827200 1665277200 26.5% completed
1665277200 1665727200 27.6% completed
166

0it [00:00, ?it/s]

[X] Please install TA-Lib to use 2crows. (pip install TA-Lib)
[X] Please install TA-Lib to use 3blackcrows. (pip install TA-Lib)
[X] Please install TA-Lib to use 3inside. (pip install TA-Lib)
[X] Please install TA-Lib to use 3linestrike. (pip install TA-Lib)
[X] Please install TA-Lib to use 3outside. (pip install TA-Lib)
[X] Please install TA-Lib to use 3starsinsouth. (pip install TA-Lib)
[X] Please install TA-Lib to use 3whitesoldiers. (pip install TA-Lib)
[X] Please install TA-Lib to use abandonedbaby. (pip install TA-Lib)
[X] Please install TA-Lib to use advanceblock. (pip install TA-Lib)
[X] Please install TA-Lib to use belthold. (pip install TA-Lib)
[X] Please install TA-Lib to use breakaway. (pip install TA-Lib)
[X] Please install TA-Lib to use closingmarubozu. (pip install TA-Lib)
[X] Please install TA-Lib to use concealbabyswall. (pip install TA-Lib)
[X] Please install TA-Lib to use counterattack. (pip install TA-Lib)
[X] Please install TA-Lib to use darkcloudcover. (pip instal

123it [00:10, 11.46it/s]


'RangeIndex' object has no attribute 'to_period'
ETHUSDT Features Generated and saved


In [10]:
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute


df = concatenated_df
df["id"] = df.index
df = df.melt(id_vars="id", var_name="time").sort_values(["id", "time"]).reset_index(drop=True)
X = extract_features(df[df["id"] < 500], column_id="id", column_sort="time", impute_function=impute)


# exp = ClassificationExperiment()
# dataset_path = '../../data/'
# data = pd.read_csv(f'{dataset_path}combined_dataset.csv')
# data.replace([np.inf, -np.inf], np.nan, inplace=True)
# s = setup(data, target = 'signal', categorical_features=['coin'], session_id = 123, use_gpu=True)
# model = create_model(LGBMClassifier())
# validation_scores = pull()
# accuracy_mean = validation_scores['Accuracy']['Mean']

# # save pipeline
# model_name = 'combined_model_tsfresh'
# save_model(model, f'../../models/{model_name}-{accuracy_mean}')
# print(f'{model_name} model saved. accuracy_mean={accuracy_mean}')
# plot_model(model, plot = 'confusion_matrix', plot_kwargs = {'percent': True})
# plot_model(model, plot = 'feature_all')


ValueError: Column must not contain NaN values: value

In [9]:
df

,id,time,value
0,0,ABER_ATR_5_15,8.919736
1,0,ABER_SG_5_15,1959.740403
2,0,ABER_XG_5_15,1941.900931
3,0,ABER_ZG_5_15,1950.820667
4,0,ACCBL_20,NaN
...,...,...,...
1164715,5519,coin,ETHUSDT
1164716,5519,high_Z_30_1,1.531344
1164717,5519,low_Z_30_1,0.119526
1164718,5519,open_Z_30_1,-0.436634


In [ ]:
from pycaret.classification import load_model, predict_model
import pandas as pd

data = pd.read_csv('../../data/combined_dataset.csv')
s = setup(data, target = 'signal', session_id = 123, use_gpu=True)

# Load trained Pipeline
model_name = 'combined_model_tsfresh'
model = load_model(f'../../models/{model_name}')
plot_model(model, plot = 'confusion_matrix', plot_kwargs = {'percent': True})
# plot_model(model, plot = 'feature_all')